# Browse AiiDA Provenance Graph

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.data.folder import FolderData
from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output
import nglview

import urlparse
from collections import OrderedDict
from pprint import pformat
import ase.io
from base64 import b64encode

In [ ]:
class StackEntry(ipw.VBox):
    def close(self):
        self.inp_links.close()
        self.html.close()
        super(StackEntry, self).close()
        
    def on_change(self, c):  
        # remove all entries below me
        myidx = self.stack_list.children.index(self)
        nentries = len(self.stack_list.children)
        for i in range(nentries-1, myidx, -1):
            self.stack_list.children[i].close()
        self.stack_list.children = self.stack_list.children[:myidx+1]
        
        # find next node
        if self.inp_links.value == None:
            return
        next_node = self.inp_links.value
        
        # add a new entry to the stack
        new_entry = StackEntry(self.stack_list, next_node)
        self.stack_list.children += (new_entry,)

        
    def __init__(self, stack_list, node):
        self.node = node
        self.stack_list = stack_list
        self.links = OrderedDict()
        self.links["Select a link"] = None
        for i, kv in enumerate(node.get_inputs(also_labels=True)):
            self.links["in:%s [%d]"%(kv[0],i)] = kv[1]
        
        self.links[50*"-"] = None
        for i, kv in enumerate(node.get_outputs(also_labels=True)):
            self.links["out:%s [%d]"%(kv[0],i)] = kv[1]
        self.inp_links = ipw.Dropdown(options=self.links)
        
        self.inp_links.observe(self.on_change, "value")
        output = "<ul>" #<h4>"+html.escape(repr(self.node))+"</h4><br>"
        pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
        output += "<li><b>class:</b> "+self.node.__class__.__name__+"</li>"
        output += "<li><b>pk:</b> "+str(self.node.pk)+"</li>"
        output += "<li><b>uuid:</b> "+self.node.uuid+"</li>"
        output += "<li><b>ctime:</b> "+str(self.node.ctime)+"</li>"
        output += "<li><b>mtime:</b> "+str(self.node.mtime)+"</li>"
        output += "<li><b>label:</b> "+self.node.label+"</li>"
        output += "<li><b>description:</b> "+self.node.description+"</li>"
        output += "<li><b>Attributes:</b>"+pre_tag+pformat(self.node.get_attrs())+"</pre></li>"
        output += "<li><b>Extras:</b>"+pre_tag+pformat(self.node.get_extras())+"</pre></li>"
        if self.node.__class__.__name__ == 'StructureData':
            structure_uuid = self.node.uuid
            structure_ase = self.node.get_ase()
            thumbnail = render_thumbnail(structure_ase)
            output += '<li><img width="100px" src="data:image/png;base64,%s" title=""></li>' % thumbnail
            output += '<li><b>Structure:</b> <a href="../surfaces/export_structure.ipynb?uuid=%s">Export Structure</a></li>'%structure_uuid
        output += "<li><b>Comments:</b>"+pre_tag+pformat(self.node.get_comments())+"</pre></li>"
        if isinstance(self.node, FolderData):
            relpath = self.node.folder.abspath.replace("/project/", "")
            folderlist = sorted(self.node.get_folder_list())
            output += "<li><b>Folder List:</b>"+pre_tag+pformat(folderlist)+"</pre></li>"
            output += "<a target='_blank' href='../../../tree/%s/path'>Browser Folder</a><br>"%relpath
        output += "<li><b>Links:</b>"
        output += "</ul>"
        
        self.html = ipw.HTML(output)
        self.border = '2px solid black'
        self.margin="20px 0 20px 0"
        super(StackEntry, self).__init__([self.html, self.inp_links,])

In [ ]:
from tempfile import NamedTemporaryFile
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
def on_submit(b):
    global stack_list
    with output:
        clear_output()
        node = load_node(int(inp_pk.value))
        for c in stack_list.children:
            c.close()
        first_entry = StackEntry(stack_list, node)
        stack_list.children = [first_entry]

btn_sub = ipw.Button(description="Load")
inp_pk = ipw.Text(description="Node PK: ")
inp_pk.on_submit(on_submit)
btn_sub.on_click(on_submit)
output = ipw.Output()

display(inp_pk, btn_sub, output)

In [ ]:
stack_list = ipw.VBox()
display(stack_list)

In [ ]:
# parse URL
url = urlparse.urlsplit(jupyter_notebook_url)
querypart = urlparse.parse_qs(url.query)
if 'pk' in querypart.keys():
    inp_pk.value = querypart['pk'][0]
    on_submit(None)